In [1]:
%pylab inline
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from collections import namedtuple
from scipy import signal
import numpy as np
from scipy.fftpack import fft,fftshift 

Populating the interactive namespace from numpy and matplotlib


In [2]:
Result = namedtuple('Result', 'subject condition')
#Event = namedtuple('Event', 'timestamp value mean') # Old
Event = namedtuple('Event', 'timestamp value mean status') # New

In [3]:
my_path='C:\\Users\\User\\Documents\\Reyhaneh\\\EEG+fMRI\\Resting\\RS\\ExportedDataLenient\\'
#my_path='C:\\MBSRtmp\\'
input_file_pattern = my_path+'*.csv'

start_time = 0   # ms
sample_interval = 4 # ms (250Hz)

Cz_channels = (1,2)
Frontal = (12,9,6,3,60) # F3,F1,Fz,F2,F4
FrontalExt = (12,9,6,3,60,59,2,8,11,13) # F3,F1,Fz,F2,F4,F6,AF4,AFz,AF3,F5
tmp = (7,8,9)
PCz_channels = (90, 121, 80)
Pz_channels = (101, 100, 129)
POz_channels = (126, 118, 127)
CL_channels = (59, 58, 65)
CR_channels = (183, 195, 182)
PL_channels = (86, 76, 85)
PR_channels = (162, 171, 172)
POL_channels = (108, 109, 117)
POR_channels = (151, 140, 139)

SiteListNames = ('Frontal','FrontalExt')
SiteListLoc = (Frontal,FrontalExt)
FreqListNames = ('theta','thetaLow','thetaHigh','alpha','beta')
FreqListRange = ((4,7), (3.5,6), (6,8.5),(8,13), (13,30))

WindowSize = 1000 # ms
OverLap = 50 # %

In [4]:
def convert_to_channel(x):
    return x +1

def time_conversion_function(start_time, sample_interval):
    def conversion(x):
        return start_time + x * sample_interval
    return conversion

In [5]:
result_map = {}

for filename in glob(input_file_pattern):
    res = Result(*filename[:-4].rsplit('_', 1))
    raw_data = pd.read_csv(filename, header=None)
    cleaned_data = raw_data.rename(index=time_conversion_function(start_time, sample_interval), columns=convert_to_channel)
    result_map[res] = cleaned_data



In [6]:
def save_figure_fft(meanwavefromFFT,fVals, experiment, channel_name):
    title_template = '{subject}|{condition}|{channel_name}'
    title = title_template.format(
        subject=experiment.subject,
        condition=experiment.condition,
        channel_name=channel_name,
    )
    filename_template = '{subject}_{condition}_{channel_name}_FFT.png'
    filename = filename_template.format(
        subject=experiment.subject,
        condition=experiment.condition,
        channel_name=channel_name,
    )
    fig, ax = plt.subplots(nrows=1, ncols=1)
    ax.plot(fVals,meanwavefromFFT)
    ax.set_title(experiment.subject[experiment.subject.rfind('\\',)+1:]+'|'+experiment.condition+'|'+channel_name)
    ax.set_xlabel('Frequency (Hz)')         
    ax.set_ylabel('|DFT Values|')
    ax.set_xlim(0,50)
    ax.set_xticks(np.arange(0, 50,10))
   # show()
   # fig = ax.get_figure()
    fig.savefig(filename)
    fig.clear()


In [7]:
def average_channel_values(result, channels):
    selected_channels = result.loc[:, channels]
    return selected_channels.mean(axis=1)

def find_maximum_and_mean_value_in_time_window(time_series, start_time, end_time):
    from matplotlib.pyplot import plot, scatter, show
           
    #return Event(timestamp=time_slice.idxmax(), value=time_slice.max(), mean=time_slice.mean()) # old
    return Event(timestamp=int(maxLocalIndOut), value=maxLocalMax, mean=time_slice.mean(), status=Status) # new

In [110]:
        indtetha = [indFreq for indFreq in range(len(FreqListNames)) if FreqListNames[indFreq]=='theta']
        indbeta = [indFreq for indFreq in range(len(FreqListNames)) if FreqListNames[indFreq]=='beta']
        indtetha
        freqindtheta = [indf for indf in range(len(fVals))  if (fVals[indf]>=FreqListRange[indtetha[0]][0] and fVals[indf]<=FreqListRange[indtetha[0]][1])]
        freqindbeta = [indf for indf in range(len(fVals))  if (fVals[indf]>=FreqListRange[indbeta[0]][0] and fVals[indf]<=FreqListRange[indbeta[0]][1])]
        print(freqindtheta)
        np.mean(meanwavefromFFT[freqindtheta])/np.mean(meanwavefromFFT[freqindbeta])
        '{:.4f}'.format(np.mean(meanwavefromFFT[freqindtheta])/np.mean(meanwavefromFFT[freqindbeta]))
        #freqindtheta = [indf for indf in range(len(fVals))  if (fVals[indf]>=FreqListRange[indtetha][0] and fVals[indf]<=FreqListRange[indtetha][1])]
    #    freqindbeta = [indf for indf in range(len(fVals))  if (fVals[indf]>=FreqListRange[indbeta][0] and fVals[indf]<=FreqListRange[indbeta][1])]
        print(result.subject[result.subject.rfind('\\',)+1:]+'|'+result.condition+'|'+SiteListNames[indSite]+
              '|'+FreqListNames[indtetha[0]]+':'+str(FreqListRange[indtetha[0]])+'/'+FreqListNames[indbeta[0]]+
              ':'+str(FreqListRange[indbeta[0]])+'|'+'{:.4f}'.format(np.mean(meanwavefromFFT[freqindtheta])/np.mean(meanwavefromFFT[freqindbeta])))
        #print(FreqListNames)#[indtetha])
        #indtetha

[529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540]
86022|RSConsv|FrontalExt|theta:(4, 7)/beta:(13, 30)|2.2651


In [8]:
NFFT=1024    
fs =250
# Hanning window of one second width and 50% overlap
#windowFilter = signal.hamming(int(WindowSize/sample_interval))
windowFilter = signal.windows.hann(int(WindowSize/sample_interval))

WindowSize = 1000
OverLap = 50
fVals = np.arange(start = -NFFT/2,stop = NFFT/2)*fs/NFFT
for result in sorted(result_map.keys()):
    for indSite in range(len(SiteListNames)):
        waveform = average_channel_values(result_map[result], SiteListLoc[indSite])
        waveformFFT=[]
        end = 0
        while (end<=len(waveform)- int(OverLap * WindowSize/100)):#len(waveform):
            start = max(0,end - int(OverLap * WindowSize/100) )
            end = start + WindowSize
            WindowIndex=(np.arange(start=start, stop=end-1,step=sample_interval ))
            waveformWindow = waveform[WindowIndex]
            waveformFFT.append(fftshift(fft(waveformWindow*windowFilter,NFFT)))
        #    print(np.arange(start=start, stop=end-1,step=sample_interval ))
        meanwavefromFFT = np.mean(np.abs(waveformFFT),axis = 0)
        save_figure_fft(meanwavefromFFT,fVals, result, SiteListNames[indSite])

        # Theta/beta:
        indtetha = [indFreq for indFreq in range(len(FreqListNames)) if FreqListNames[indFreq]=='theta']
        indbeta = [indFreq for indFreq in range(len(FreqListNames)) if FreqListNames[indFreq]=='beta']
        freqindtheta = [indf for indf in range(len(fVals))  if (fVals[indf]>=FreqListRange[indtetha[0]][0] and fVals[indf]<=FreqListRange[indtetha[0]][1])]
        freqindbeta = [indf for indf in range(len(fVals))  if (fVals[indf]>=FreqListRange[indbeta[0]][0] and fVals[indf]<=FreqListRange[indbeta[0]][1])]
        print(result.subject[result.subject.rfind('\\',)+1:]+'|'+result.condition+'|'+SiteListNames[indSite]+
              '|'+FreqListNames[indtetha[0]]+':'+str(FreqListRange[indtetha[0]])+'/'+FreqListNames[indbeta[0]]+
              ':'+str(FreqListRange[indbeta[0]])+'|'+'{:.4f}'.format(np.mean(meanwavefromFFT[freqindtheta])/np.mean(meanwavefromFFT[freqindbeta])))

        for indFreq in range(len(FreqListNames)):
            freqind = [indf for indf in range(len(fVals))  if (fVals[indf]>=FreqListRange[indFreq][0] and fVals[indf]<=FreqListRange[indFreq][1])]
            print(result.subject[result.subject.rfind('\\',)+1:]+'|'+result.condition+'|'+SiteListNames[indSite]+'|'+FreqListNames[indFreq]+':'+str(FreqListRange[indFreq])+'|'+'{:.4f}'.format(np.mean(meanwavefromFFT[freqind])))
                          


86001|RSLenn|Frontal|theta:(4, 7)/beta:(13, 30)|1.9703
86001|RSLenn|Frontal|theta:(4, 7)|104.5909
86001|RSLenn|Frontal|thetaLow:(3.5, 6)|109.4968
86001|RSLenn|Frontal|thetaHigh:(6, 8.5)|107.3512
86001|RSLenn|Frontal|alpha:(8, 13)|101.4193
86001|RSLenn|Frontal|beta:(13, 30)|53.0843
86001|RSLenn|FrontalExt|theta:(4, 7)/beta:(13, 30)|2.1288
86001|RSLenn|FrontalExt|theta:(4, 7)|84.0581
86001|RSLenn|FrontalExt|thetaLow:(3.5, 6)|87.7091
86001|RSLenn|FrontalExt|thetaHigh:(6, 8.5)|85.8393
86001|RSLenn|FrontalExt|alpha:(8, 13)|82.9797
86001|RSLenn|FrontalExt|beta:(13, 30)|39.4856
86002|RSLenn|Frontal|theta:(4, 7)/beta:(13, 30)|1.9206
86002|RSLenn|Frontal|theta:(4, 7)|227.5771
86002|RSLenn|Frontal|thetaLow:(3.5, 6)|257.3724
86002|RSLenn|Frontal|thetaHigh:(6, 8.5)|167.5645
86002|RSLenn|Frontal|alpha:(8, 13)|148.5880
86002|RSLenn|Frontal|beta:(13, 30)|118.4911
86002|RSLenn|FrontalExt|theta:(4, 7)/beta:(13, 30)|2.1090
86002|RSLenn|FrontalExt|theta:(4, 7)|161.1059
86002|RSLenn|FrontalExt|thetaLow:(3

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


86012|RSLenn|Frontal|theta:(4, 7)/beta:(13, 30)|1.5409
86012|RSLenn|Frontal|theta:(4, 7)|165.5946
86012|RSLenn|Frontal|thetaLow:(3.5, 6)|168.5238
86012|RSLenn|Frontal|thetaHigh:(6, 8.5)|155.7276
86012|RSLenn|Frontal|alpha:(8, 13)|146.5879
86012|RSLenn|Frontal|beta:(13, 30)|107.4639
86012|RSLenn|FrontalExt|theta:(4, 7)/beta:(13, 30)|1.5841
86012|RSLenn|FrontalExt|theta:(4, 7)|149.2035
86012|RSLenn|FrontalExt|thetaLow:(3.5, 6)|154.4701
86012|RSLenn|FrontalExt|thetaHigh:(6, 8.5)|135.0455
86012|RSLenn|FrontalExt|alpha:(8, 13)|132.4271
86012|RSLenn|FrontalExt|beta:(13, 30)|94.1874
86013|RSLenn|Frontal|theta:(4, 7)/beta:(13, 30)|1.8925
86013|RSLenn|Frontal|theta:(4, 7)|123.4812
86013|RSLenn|Frontal|thetaLow:(3.5, 6)|127.4277
86013|RSLenn|Frontal|thetaHigh:(6, 8.5)|113.4004
86013|RSLenn|Frontal|alpha:(8, 13)|121.4792
86013|RSLenn|Frontal|beta:(13, 30)|65.2487
86013|RSLenn|FrontalExt|theta:(4, 7)/beta:(13, 30)|1.8618
86013|RSLenn|FrontalExt|theta:(4, 7)|115.9943
86013|RSLenn|FrontalExt|thetaLo

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>